In [1]:
# Clears all of CIRCADIEM databases
#mycursor.execute("DROP TABLE main_econsent_declaration;");

In [1]:
import mysql.connector     #python library for connecting to server
import csv
import datetime

import openpyxl
import decimal

connection  = mysql.connector.connect(user='root', password='password123',host='127.0.0.1')

# Create the database and use database
mycursor = connection.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS CIRCADIEM")
connection.database = 'CIRCADIEM'

# Import Functions

In [2]:
# mycursor and connection must be initialized for this function to work. (The correct database should be connected to)
def import_database(table_name, data):
    # Get the headings from the mysql table
    sql_heading_Query = "SHOW COLUMNS FROM {}".format(table_name)
    mycursor.execute(sql_heading_Query)
    heading_data = mycursor.fetchall()
    
    sql_string_heading = "INSERT IGNORE INTO {} (".format(table_name) # INSERT IGNORE is used to avoid writing over previously inserted data points (This will ensure the timestamps are correct)
    for a in range (0, len(heading_data) - 1): # (-1) For the allowance of adding commas to the string
        sql_string_heading = sql_string_heading + heading_data[a][0] + ","
    sql_string_heading = sql_string_heading + "updated_at)"

    for a in range (0, len(data)): #Remove the headings from the dataset 
        sql_string = sql_string_heading + " VALUES ("
        for b in range (0, len(data[0])):
            if data[a][b] == "":
                sql_string = sql_string + 'NULL,'
            elif '"' in str(data[a][b]):
                sql_string = sql_string + '"' + str(data[a][b]).replace('"',"'") + '",' #This is to handle strings that contain double quotes. It will replace any double quotes with single quotes
            else:
                sql_string = sql_string + '"' + str(data[a][b]) + '",'
        sql_string = sql_string + "NOW())"
        mycursor.execute(sql_string)
    connection.commit()
    
# The function that collapses survey questions as a field rather than a label
def import_database_survey(table_name, data, questions_headings, questions_data):
    # Get the headings from the mysql table
    sql_heading_Query = "SHOW COLUMNS FROM {}".format(table_name)
    mycursor.execute(sql_heading_Query)
    heading_data = mycursor.fetchall()
    
    sql_string_heading = "INSERT IGNORE INTO {} (".format(table_name) # INSERT IGNORE is used to avoid writing over previously inserted data points (This will ensure the timestamps are correct)
    for a in range (0, len(heading_data) - 1): # (-1) For the allowance of adding commas to the string
        sql_string_heading = sql_string_heading + heading_data[a][0] + ","
    sql_string_heading = sql_string_heading + "updated_at)"
    
    for a in range (0, len(data)):
        sql_string = sql_string_heading + " VALUES ("
        for b in range (0, len(data[0])):
            if data[a][b] == "":
                sql_string = sql_string + 'NULL,'
            elif '"' in str(data[a][b]):
                sql_string = sql_string + '"' + str(data[a][b]).replace('"',"'") + '",' #This is to handle strings that contain double quotes. It will replace any double quotes with single quotes
            else:
                sql_string = sql_string + '"' + str(data[a][b]) + '",'
                
        for c in range (0, len(questions_data[a])):
            sql_string_collapsed = sql_string

            # Add the question number
            sql_string_collapsed = sql_string_collapsed + '"' + str(c + 1) + '",'
            
            temp_question_heading = questions_headings[c]
            if temp_question_heading == "":
                sql_string_collapsed = sql_string_collapsed + 'NULL,'
            elif '"' in str(temp_question_heading):
                sql_string_collapsed = sql_string_collapsed + '"' + str(temp_question_heading).replace('"',"'") + '",' #This is to handle strings that contain double quotes. It will replace any double quotes with single quotes
            else:
                sql_string_collapsed = sql_string_collapsed + '"' + str(temp_question_heading) + '",'

            temp_question_answer = questions_data[a][c]
            if temp_question_answer == "":
                sql_string_collapsed = sql_string_collapsed + 'NULL,'
            elif '"' in str(temp_question_answer):
                sql_string_collapsed = sql_string_collapsed + '"' + str(temp_question_answer).replace('"',"'") + '",' #This is to handle strings that contain double quotes. It will replace any double quotes with single quotes
            else:
                sql_string_collapsed = sql_string_collapsed + '"' + str(temp_question_answer) + '",'
            sql_string_collapsed = sql_string_collapsed + "NOW())"
            #print(sql_string_collapsed)
            # if table_name == "fu_sleep_diary":
            #     print(sql_string_collapsed)
            mycursor.execute(sql_string_collapsed)
    connection.commit()

# CIRCADIEM Hospital

In [5]:
# Export file from Redcap
# Data Exports, Reports, and Stats -> All data (Export data) -> check CSV / Microsoft Excel (RAW) export format -> * do not check deitentified objects
# -> Export data

# Read the data file for import
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_LABELS_2023-08-16_1125.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_LABELS_2023-11-01_1327.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_LABELS_2024-05-17_1104.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_LABELS_2024-07-24_1055.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_LABELS_2024-07-26_1311.csv" 
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_2024-07-31_1201.csv" # The cut of data used for DSMC / Interim Report
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_2024-08-06_1755.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_2024-08-14_1126.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_2024-08-19_1031.csv"
# filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_2024-08-26_1803.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_2024-10-31_1245.csv"


# The format of hospital 5859 is different to CIRCADIEM Hospital.
# Will manually import 5859 for now.
#filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital_5859/CIRCADIEMHospital585_DATA_LABELS_2024-07-26_1312.csv"

data = []
with open(r'{}'.format(filepath)) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        data.append(row)

## Create SQL Tables

In [6]:
# Check to see if the fields in the Hospital Instruments match those defined by the data schema.
Hospital_Instruments_Survey = ["preg_surv", "infection_econsent", "mri_econsent", "main_econsent", "main_econsent_declaration", "sleep_econsent"]
Hospital_Instruments = ["screen", "main_pis", "main_econsent", "main_econsent_declaration", "enrol", "randomise", "contacts", "an_hx", "birth_hx", "preg_surv", "infection_pis", "infection_econsent", "mri_pis", "mri_econsent", "sleep_pis", "sleep_econsent", "growth", "pea_pod", "light_audit", "sound_audit", "36w_oximetry_obs", "substudies", "cardioresp", "headus", "mri", "neomorbidity", "discharge", "survival", "adverse_event", "protocol_deviation", "withdrawal", "report_calculations"]

# View all the column numbers which link to "Complete?" heading which indicates the end of a instrument's data
Column_list = [8] # Starting after the last instrument
for a in range (0 ,len(data[0])):
    temp = data[0][a]
    if "_" in temp:
        temp = temp.split("_")[-1]
    if temp == "complete": # This is the last column for the Instrument
        Column_list.append(a + 1)

for a in range (0, len(Hospital_Instruments)):
    
    table_name = Hospital_Instruments[a]
    sql_string = """DESCRIBE {}""".format(table_name)
    mycursor.execute(sql_string)
    temp_data = mycursor.fetchall()

    # Column 0 = Auto ID, Column 1 = Event_Name
#     table_name = Hospital_Instruments[a]
#     print ("!!!{},{}!!!".format(a, table_name))
#     for b in range (Column_list[a], Column_list[a + 1]):
#         print(data[0][b])

#     print("----------------------------------")

    # Column 0 = Auto ID, Column 1 = Event_Name
    if table_name not in Hospital_Instruments_Survey:
            print ("!!!{},{}!!!".format(a, table_name))
            sql_field_count = 2
            for b in range (Column_list[a], Column_list[a + 1]):
                print(data[0][b],  "------------", temp_data[sql_field_count][0])
                sql_field_count = sql_field_count + 1

            print("----------------------------------")
            
    else:
        print ("!!!{},{}!!!".format(a, table_name))
        print("----------------------------------")

!!!0,screen!!!
dob ------------ inborn_yn
edd ------------ sex
ga_calc ------------ b_dtm
ga_strata ------------ dob
rct_other_yn ------------ edd_dtm
rct_other___1 ------------ edd
rct_other___2 ------------ ga_calc
rct_other___3 ------------ ga_cat_calc
rct_other___4 ------------ ga_strata
rct_other___5 ------------ rct_other_yn
rct_other___6 ------------ rct_other___1
rct_other___7 ------------ rct_other___2
rct_other___8 ------------ rct_other___3
rct_other___99 ------------ rct_other___4
rct_other___ni ------------ rct_other___5
rct_other___unk ------------ rct_other___6
rct_other___nask ------------ rct_other___7
rct_other___asku ------------ rct_other___8
rct_other___na ------------ rct_other___99
rctother_name ------------ rct_other___ni
other_acr ------------ rct_other___unk
other_neuro_yn ------------ rct_other___nask
cg_neuro ------------ rct_other___asku
cong_abnorm_text ------------ rct_other___na
crit_ill ------------ rctother_name
dischhosp_exp ------------ other_acr
fu2

IndexError: list index out of range

In [287]:
#mycursor.execute("DROP TABLE enrol")

### Screen

In [288]:
# Screen
mycursor.execute("""CREATE TABLE IF NOT EXISTS screen (
Auto_ID VARCHAR (8),
Event_Name TEXT,
inborn_yn TEXT,
sex INT,
b_dtm DATETIME,
dob DATE,
edd_dtm DATETIME,
edd DATE,
ga_calc TEXT,
ga_cat_calc TEXT,
ga_strata TEXT,
rct_other_yn TEXT,
rct_other___1 TEXT,
rct_other___2 TEXT,
rct_other___3 TEXT,
rct_other___4 TEXT,
rct_other___5 TEXT,
rct_other___6 TEXT,
rct_other___7 TEXT,
rct_other___8 TEXT,
rct_other___99 TEXT,
rct_other___ni TEXT,
rct_other___unk TEXT,
rct_other___nask TEXT,
rct_other___asku TEXT,
rct_other___na TEXT,
rctother_name TEXT,
other_acr TEXT,
other_neuro_yn TEXT,
cg_neuro TEXT,
cong_abnorm_text TEXT,
crit_ill TEXT,
dischhosp_exp TEXT,
fu2y_yn TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Main PIS

In [289]:
# Main_PIS
mycursor.execute("""CREATE TABLE IF NOT EXISTS main_pis (
Auto_ID VARCHAR (8),
Event_Name TEXT,
main_pis_timestamp DATETIME,
pisread_yn TEXT,
pisend_opt TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Main Econsent - Survey

In [290]:
# Main Econsent - Survey
mycursor.execute("""CREATE TABLE IF NOT EXISTS main_econsent (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### Main Econsent Declaration

In [291]:
# Main econsent declaration
mycursor.execute("""CREATE TABLE IF NOT EXISTS main_econsent_declaration (
Auto_ID VARCHAR (8),
Event_Name TEXT,
main_econsent_declaration_timestamp DATETIME, 
consentdecl_name2 TEXT,
consentdecl_sig2 TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Enrol

In [292]:
# Enrol
mycursor.execute("""CREATE TABLE IF NOT EXISTS enrol (
Auto_ID VARCHAR (8),
Event_Name TEXT,
par_approach_yn TEXT,
approach_reason___1 TEXT,
approach_reason___2 TEXT,
approach_reason___3 TEXT,
approach_reason___4 TEXT,
approach_reason___5 TEXT,
approach_reason___6 TEXT,
approach_reason___7 TEXT,
approach_reason___8 TEXT,
approach_reason___99 TEXT,
approach_reason___ni TEXT,
approach_reason___unk TEXT,
approach_reason___nask TEXT,
approach_reason___asku TEXT,
approach_reason___na TEXT,
approach_other TEXT,
par_consent TEXT,
consent_type TEXT,
enrol_dtm DATETIME,
age_enrol TEXT,
resid_mainconsent TEXT,
consent_upload TEXT,
consent_fname TEXT,
consent_form TEXT,
par_signform_yn TEXT,
par_consentcopy_dt TEXT,
consent_mr_yn TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Randomise

In [293]:
# Randomise
mycursor.execute("""CREATE TABLE IF NOT EXISTS Randomise (
Auto_ID VARCHAR (8),
Event_Name TEXT,
multiple TEXT,
birthord TEXT,
mult_rand TEXT,
firstborn_id TEXT,
confirm TEXT,
confirm_twin TEXT,
rand_gp TEXT,
nonrand_gp TEXT,
study_id TEXT,
rand_dtm DATETIME,
Complete TEXT,
updated_at TIMESTAMP)""")

### Contacts

In [294]:
# Contacts
mycursor.execute("""CREATE TABLE IF NOT EXISTS contacts (
Auto_ID VARCHAR (8),
Event_Name TEXT,
indig_yn TEXT,
m_ethnic TEXT,
p2_yn TEXT,
p2_ethnic TEXT,
p2_sameaddress_yn TEXT,
p2_state TEXT,
alt_type TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### An Hx

In [295]:
mycursor.execute("""CREATE TABLE IF NOT EXISTS An_Hx (
Auto_ID VARCHAR (8),
Event_Name TEXT,
m_lmp_dt DATE,
mans_dt DATE,
mprob_yn TEXT,
mprob_type___1 TEXT,
mprob_type___2 TEXT,
mprob_type___3 TEXT,
mprob_type___4 TEXT,
mprob_type___5 TEXT,
mprob_type___6 TEXT,
mprob_type___7 TEXT,
mprob_type___ni TEXT,
mprob_type___unk TEXT,
mprob_type___nask TEXT,
mprob_type___asku TEXT,
mprob_type___na TEXT,
thyr_med TEXT,
chor_clin_yn TEXT,
chor_clin_basis___1 TEXT,
chor_clin_basis___2 TEXT,
chor_clin_basis___3 TEXT,
chor_clin_basis___4 TEXT,
chor_clin_basis___5 TEXT,
chor_clin_basis___98 TEXT,
chor_clin_basis___99 TEXT,
chor_clin_basis___ni TEXT,
chor_clin_basis___unk TEXT,
chor_clin_basis___nask TEXT,
chor_clin_basis___asku TEXT,
chor_clin_basis___na TEXT,
chor_hist TEXT,
plac_histo_fname TEXT,
plac_histo_file TEXT,
romdtm TEXT,
romdurn TEXT,
ans TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Birth Hx

In [296]:
# Birth Hx
mycursor.execute("""CREATE TABLE IF NOT EXISTS birth_hx (
Auto_ID VARCHAR (8),
Event_Name TEXT,
b_hc TEXT,
bw TEXT,
bwz TEXT,
bwcent TEXT,
iugr TEXT,
delmode TEXT,
as1 TEXT,
as5 TEXT,
as10 TEXT,
cordabg_yn TEXT,
abgcord_source TEXT,
cord_ph TEXT,
cord_pco2 TEXT,
cord_be TEXT,
cord_lact TEXT,
msaf_yn TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Preg Survey - Survey

In [297]:
# Preg Surv - Survey
mycursor.execute("""CREATE TABLE IF NOT EXISTS preg_surv (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### Infection PIS

In [298]:
# Infection PIS
mycursor.execute("""CREATE TABLE IF NOT EXISTS infection_pis (
Auto_ID VARCHAR (8),
Event_Name TEXT,
infection_pis_timestamp DATETIME,
iipisend_opt TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Infection eConsent - Survey

In [299]:
# Infection eConsent - Survey
mycursor.execute("""CREATE TABLE IF NOT EXISTS infection_econsent (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### MRI PIS

In [300]:
# MRI PIS
mycursor.execute("""CREATE TABLE IF NOT EXISTS mri_pis (
Auto_ID VARCHAR (8),
Event_Name TEXT,
mri_pis_timestamp DATETIME,
mripisend_opt TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### MRI eConsent - Survey

In [301]:
# MRI eConsent - Survey
mycursor.execute("""CREATE TABLE IF NOT EXISTS mri_econsent (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### Sleep PIS

In [302]:
# Sleep PIS
mycursor.execute("""CREATE TABLE IF NOT EXISTS sleep_pis (
Auto_ID VARCHAR (8),
Event_Name TEXT,
sleep_pis_timestamp DATETIME,
sleeppisend_opt TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Sleep eConsent - Survey

In [303]:
# Sleep eConsent - Survey
mycursor.execute("""CREATE TABLE IF NOT EXISTS sleep_econsent (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### Growth

In [304]:
# Growth information
mycursor.execute("""CREATE TABLE IF NOT EXISTS growth (
Auto_ID VARCHAR (8),
Event_Name TEXT,
growth_timestamp DATETIME,
growth_dtm DATETIME,
cpna_growth TEXT,
crown_heel TEXT,
length_cent TEXT,
length_z TEXT,
head_circ TEXT,
hc_cent TEXT,
hc_z TEXT,
wt TEXT,
wt_cent TEXT,
wt_z TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Pea pod

In [305]:
# Pea pod
mycursor.execute("""CREATE TABLE IF NOT EXISTS pea_pod (
Auto_ID VARCHAR (8),
Event_Name TEXT,
ppod_yn TEXT,
noppod___1 TEXT,
noppod___2 TEXT,
noppod___3 TEXT,
noppod___99 TEXT,
noppod___ni TEXT,
noppod___unk TEXT,
noppod___nask TEXT,
noppod___asku TEXT,
noppod___na TEXT,
ppod_dt DATE,
ppod_res_id TEXT,
f_perc TEXT,
ff_perc TEXT,
fmass_kg TEXT,
ffmass_kg TEXT,
bmass_kg TEXT,
bvol_l TEXT,
bdens TEXT,
fmass_dens TEXT,
ffmass_dens TEXT,
bsurf_cm2 TEXT,
thorax_vol_l TEXT,
peapod_fname TEXT,
peapod_file TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

In [306]:
### Light Audit

In [307]:
# Light Audit
mycursor.execute("""CREATE TABLE IF NOT EXISTS light_audit (
Auto_ID VARCHAR (8),
Event_Name TEXT,
light_yn TEXT,
light_reason TEXT,
light_dt DATE,
light_fname TEXT,
light_file TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

In [308]:
### Sound Audit

In [309]:
# Sound Audit
mycursor.execute("""CREATE TABLE IF NOT EXISTS sound_audit (
Auto_ID VARCHAR (8),
Event_Name TEXT,
sound_yn TEXT,
sound_reason TEXT,
sound_dt DATE,
sound_fname TEXT,
sound_file TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### 36w_Oximetry_obs

In [310]:
# 36w_Oximetry_obs
mycursor.execute("""CREATE TABLE IF NOT EXISTS 36w_Oximetry_obs (
Auto_ID VARCHAR (8),
Event_Name TEXT,
shiftdtm DATETIME,
shift_supp___1 TEXT,
shift_supp___2 TEXT,
shift_supp___3 TEXT,
shift_supp___4 TEXT,
shift_supp___5 TEXT,
shift_supp___ni TEXT,
shift_supp___unk TEXT,
shift_supp___nask TEXT,
shift_supp___asku TEXT,
shift_supp___na TEXT,
shift_o2 TEXT,
shift_hhf_fl TEXT,
shift_cpap_press TEXT,
shift_hb TEXT,
shift_ort TEXT,
shiftfio2 TEXT,
shiftspo2 TEXT,
altitude TEXT,
altitudecalc TEXT,
shift_fname TEXT,
shift_file TEXT,
spo2_24av TEXT,
fi02_24h TEXT,
shift_pred TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Substudies

In [311]:
# Substudies
mycursor.execute("""CREATE TABLE IF NOT EXISTS Substudies (
Auto_ID VARCHAR (8),
Event_Name TEXT,
biol_yn TEXT,
biol_res_id TEXT,
biol_consent_dt DATE,
biol_fname TEXT,
biol_consent_file TEXT,
physiol_yn TEXT,
physiol_res_id TEXT,
physiol_consent_dt DATE,
physiol_fname TEXT,
physiol_file TEXT,
mri_yn TEXT,
mri_res_id TEXT,
mri_consent_dt DATE,
mri_fname TEXT,
mri_file TEXT,
sleep_yn TEXT,
sleep_res_id TEXT,
sleep_consent_dt DATE,
sleep_fname TEXT,
sleep_consent_file TEXT,
behav_yn TEXT,
behav_res_id TEXT,
behav_consent_dt DATE,
behav_fname TEXT,
behav_file TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Cardioresp

In [312]:
# Cardioresp
mycursor.execute("""CREATE TABLE IF NOT EXISTS Cardioresp (
Auto_ID VARCHAR (8),
Event_Name TEXT,
cardioresp_timestamp DATETIME,
resp_diag TEXT,
surfact_yn TEXT,
surfact_num TEXT,
aleak_yn TEXT,
no_yn TEXT,
no_hr TEXT,
ecmo_yn TEXT,
hiflo_hr TEXT,
cpaphr TEXT,
nventhr TEXT,
ippvhr TEXT,
hfovhr TEXT,
lastrespsupp_dtm DATETIME,
respsupp36wk TEXT,
systcscld TEXT,
o2hr TEXT,
pda_yn TEXT,
pdarx___0 TEXT,
pdarx____1 TEXT,
pdarx____2 TEXT,
pdarx___ni TEXT,
pdarx___unk TEXT,
pdarx___nask TEXT,
pdarx___asku TEXT,
pdarx___na TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Headus

In [313]:
# Headus
mycursor.execute("""CREATE TABLE IF NOT EXISTS Headus (
Auto_ID VARCHAR (8),
Event_Name TEXT,
uss_yn TEXT,
uss_dtm DATETIME,
left_ivh TEXT,
right_ivh TEXT,
cerebral_cysts TEXT,
left_cysts TEXT,
right_cysts TEXT,
uss_fname TEXT,
uss_file TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### MRI

In [314]:
# MRI
mycursor.execute("""CREATE TABLE IF NOT EXISTS MRI (
Auto_ID VARCHAR (8),
Event_Name TEXT,
mri TEXT,
mri_dtm DATETIME,
Complete TEXT,
updated_at TIMESTAMP)""")

### Neomorbidity

In [315]:
# Neomorbidity
mycursor.execute("""CREATE TABLE IF NOT EXISTS Neomorbidity (
Auto_ID VARCHAR (8),
Event_Name TEXT,
neomorbidity_timestamp DATETIME,
necdx TEXT,
necdx_dtm DATETIME,
necrx___0 TEXT,
necrx___1 TEXT,
necrx___2 TEXT,
necrx___ni TEXT,
necrx___unk TEXT,
necrx___nask TEXT,
necrx___asku TEXT,
necrx___na TEXT,
necstage TEXT,
tpn TEXT,
tpn_hr TEXT,
tpn_d TEXT,
pncease_dtm DATETIME,
enteral_120 TEXT,
bmdischarge TEXT,
rop TEXT,
rop_plus_yn TEXT,
roprx TEXT,
rop_vegf TEXT,
sepsis_proven_yn TEXT,
sepsis_proven_no TEXT,
photo_rx TEXT,
sbr_max TEXT,
sbr_dtm DATETIME,
Complete TEXT,
updated_at TIMESTAMP)""")

In [316]:
### Discharge

In [317]:
# Discharge
mycursor.execute("""CREATE TABLE IF NOT EXISTS discharge (
Auto_ID VARCHAR (8),
Event_Name TEXT,
discharge_timestamp DATETIME,
dcgdtm DATETIME,
dcg_origin TEXT,
inthosp TEXT,
otherhosp TEXT,
dcg_origin2 TEXT,
dcgdtm2 TEXT,
inthosp3 TEXT,
otherhosp3 TEXT,
dcg_origin3 TEXT,
dcgdtm3 TEXT,
inthosp4 TEXT,
otherhosp4 TEXT,
dcg_origin4 TEXT,
dcgdtm4 TEXT,
inthosp5 TEXT,
otherhosp5 TEXT,
dcgdtm5 TEXT,
dischhome_dtm DATETIME,
hosp_durn TEXT,
disch_comment TEXT,
ceased_o2_yn TEXT,
o2cease_dtm DATETIME,
Complete TEXT,
updated_at TIMESTAMP)""")

### Survival

In [318]:
# Survival
mycursor.execute("""CREATE TABLE IF NOT EXISTS survival (
Auto_ID VARCHAR (8),
Event_Name TEXT,
dth_yn TEXT,
dth_dtm DATETIME,
dth_cause TEXT,
died_age TEXT,
died_pma TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Adverse Events

In [319]:
# Adverse Events
mycursor.execute("""CREATE TABLE IF NOT EXISTS adverse_event (
Auto_ID VARCHAR (8),
Event_Name TEXT,
ae_yn TEXT,
ae_yn_2 TEXT,
ae_note TEXT,
ae_dtm DATETIME,
ae_end_dt DATE,
ae_severity TEXT,
ae_related TEXT,
ae_device___1 TEXT,
ae_device___2 TEXT,
ae_device___ni TEXT,
ae_device___unk TEXT,
ae_device___nask TEXT,
ae_device___asku TEXT,
ae_device___na TEXT,
ae_sequelae TEXT,
ae_action TEXT,
ae_type___1 TEXT,
ae_type___2 TEXT,
ae_type___3 TEXT,
ae_type___ni TEXT,
ae_type___unk TEXT,
ae_type___nask TEXT,
ae_type___asku TEXT,
ae_type___na TEXT,
sae_yn TEXT,
ae_skin TEXT,
ae_infection TEXT,
ae_safety TEXT,
otherae_yn TEXT,
ae_other TEXT,
aecode1 TEXT,
aecode2 TEXT,
aecode3 TEXT,
other_sae_yn TEXT,
ae_file TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Protocol Deviation

In [320]:
# Protocol Deviation
mycursor.execute("""CREATE TABLE IF NOT EXISTS protocol_deviation (
Auto_ID VARCHAR (8),
Event_Name TEXT,
devyn TEXT,
devdt DATE,
idevdt DATE,
devnotes TEXT,
devcode TEXT,
devcodeoth TEXT,
devae TEXT,
devfin TEXT,
devfindt DATE,
devethics TEXT,
devethicsdt DATE,
devsign TEXT,
devpdf TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Withdrawal

In [321]:
# Withdrawal
mycursor.execute("""CREATE TABLE IF NOT EXISTS withdrawal (
Auto_ID VARCHAR (8),
Event_Name TEXT,
withdraw_yn TEXT,
withdraw_dt DATE,
withdraw_form TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Report Calculations

In [322]:
# Report Calculations
mycursor.execute("""CREATE TABLE IF NOT EXISTS report_calculations (
Auto_ID VARCHAR (8),
Event_Name TEXT,
hosp_durn_1 TEXT,
hosp_durn_2 TEXT,
inthosp2_copy TEXT,
hosp_durn_3 TEXT,
inthosp3_copy TEXT,
hosp_durn_4 TEXT,
inthosp4_copy TEXT,
hosp_durn_home TEXT,
dischcalc1 TEXT,
dth_calc TEXT,
withdraw_calc TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

## Import Data

In [323]:
# Initial 6 columns refer to linking fields
# ï»¿"Auto_ID"
# Event_Name
# Repeat_Instrument
# Repeat_Instance
# Data_Access_Group
# Survey_Identifier

# View all the column numbers which link to "Complete?" heading which indicates the end of a instrument's data
Column_list = [8] # Starting after the last instrument
for a in range (0 ,len(data[0])):
    temp = data[0][a]
    if "_" in temp:
        temp = temp.split("_")[-1]
    if temp == "complete": # This is the last column for the Instrument
        Column_list.append(a + 1)

event_name_list = ["d03_arm_1", "d7_arm_1", "d14_arm_1", "d28_arm_1", "d56_arm_1", "36w_arm_1", "disch_arm_1", "protocol_deviation_arm_1", "study_withdrawal_arm_1", "survival_arm_1"]  

# Manually create list for each event type that triggers an instrument
D03 = ["screen", "main_pis", "main_econsent", "main_econsent_declaration", "enrol", "randomise", "contacts", "an_hx", "birth_hx", "preg_surv", "growth", "headus", "report_calculations"]
D7 = ["infection_pis", "infection_econsent", "growth", "headus"]
D14 = ["growth"]
D28 = ["growth", "headus"]
D56 = ["growth"]
W36 = ["infection_pis", "infection_econsent", "mri_pis", "mri_econsent", "sleep_pis", "sleep_econsent", "growth", "pea_pod", "36w_oximetry_obs", "headus"]
Disch = ["growth", "substudies", "cardioresp", "mri", "neomorbidity", "discharge"]
Protocol_deviation = ["protocol_deviation"]
Study_withdrawal = ["withdrawal"]
Survival = ["survival"]
         
event_name_mapping = [D03, D7, D14, D28, D56, W36, Disch, Protocol_deviation, Study_withdrawal, Survival]
       
# The name of the filter for the repeat Instruments
event_name_list_repeatInstrument = ["light_audit", "sound_audit", "adverseevent"]

# Create a dictionary with the Hospital Instruments / If it is a survey / Starting Column / Ending Column / Event to Instrument Mapping / Repeat_Instruments

# Tables are stored into the database based on whether they are a survey or not (Manually determined)
# 1 if Survey
# 0 if Not Survey

# Manually deduce whether or not the instrument should be stored in a survey structure
Hospital_Instruments_Survey = ["main_econsent", "preg_surv", "infection_econsent", "mri_econsent", "sleep_econsent"]

# The datarows are filtered either based on event_name or repeat_instrument. This will display the right data row when the data should be present
# 1 if filtered by Repeat Instrument
# 0 if filtered by Event Name

# Data rows that need to be filtered by repeat_instruments rather than event_name
Repeat_Instruments = ["light_audit", "sound_audit", "adverse_event"]

NoImport_List = ["240", "237"]

Hospital_Dictionary = {}
# Example screen: [0, 6, 41] -> instrument: "screen", not survey, first column: 6, last column: 41
for a in range (0, len(Hospital_Instruments)):
    table_name = Hospital_Instruments[a]
    
    temp_array = []
    # Check to see if the instrument is a survey
    if table_name in Hospital_Instruments_Survey:
         temp_array.append(1)
    else:
         temp_array.append(0)
         
    # First column and Last column of data
    temp_array.append(Column_list[a])
    temp_array.append(Column_list[a+1])
         
    # Check to see if the instrument needs to be filtered by Repeat Instrument
    if table_name in Repeat_Instruments:
        temp_array.append(1)
         # Append an array of the repeat_instrument strings that it need to be filtered on
        temp_list = []
        for b in range (0, len(Repeat_Instruments)):
            if Repeat_Instruments[b] == table_name:
                temp_list.append(event_name_list_repeatInstrument[b])
        temp_array.append(temp_list)
    else:
        temp_array.append(0)
        # Append an array of the event_names that need to be filtered on
        temp_list = []
        for b in range (0, len(event_name_mapping)):
            if table_name in event_name_mapping[b]:
                temp_list.append(event_name_list[b])
        temp_array.append(temp_list)
        
    # Create Dictionary
    Hospital_Dictionary[table_name] = temp_array

In [324]:
#Hospital_Dictionary

In [325]:
table_name = "protocol_deviation"

# Clear the data from the table
sql_string = "TRUNCATE TABLE {}".format(table_name)
mycursor.execute(sql_string)

print("{} truncated".format(table_name))

Survey_binary = Hospital_Dictionary[table_name][0]
Column1 = Hospital_Dictionary[table_name][1]
Column2 = Hospital_Dictionary[table_name][2]
Repeat_Instrument_bin = Hospital_Dictionary[table_name][3]
StringList = Hospital_Dictionary[table_name][4]


# If the Instrument is NOT a survey
if Hospital_Dictionary[table_name][0] == 0:

    # Retrieve the appropriate columns and rows
    import_data = []
    for a in range (0, len(data)):
        auto_id = data[a][0]
        event_name = data[a][1]
        repeat_instrument = data[a][2]

        # Check to see if the data is filtered by repeat instrument or event_name
        if auto_id not in NoImport_List:#Do not import the data for the two participants from Hospital5859
            if Repeat_Instrument_bin == 1:
                if repeat_instrument in StringList:
                    temp_data = data[a][0:2] + data[a][Column1:Column2]
                    import_data.append(temp_data)
            else:
                if event_name in StringList and repeat_instrument == "protocol_deviation":
                    temp_data = data[a][0:2] + data[a][Column1:Column2]
                    import_data.append(temp_data)

    # Check to see if the final columne heading is "Comlete?"
    if data[0][Column2 - 1][-8:] != "complete":
        print("Error: Last Column was not 'complete' field")

    # Import the data into the correct table in the database
    import_database(table_name, import_data)
    print("{} imported".format(table_name))

protocol_deviation truncated
protocol_deviation imported


In [329]:
# Iterate through the dictionary of Instruments, Check to see if they are survey / Check to see what the rows are filtered on / Import to SQL

for table_name in Hospital_Dictionary:
    
    # Clear the data from the table
    sql_string = "TRUNCATE TABLE {}".format(table_name)
    mycursor.execute(sql_string)
    
    print("{} truncated".format(table_name))
    
    Survey_binary = Hospital_Dictionary[table_name][0]
    Column1 = Hospital_Dictionary[table_name][1]
    Column2 = Hospital_Dictionary[table_name][2]
    Repeat_Instrument_bin = Hospital_Dictionary[table_name][3]
    StringList = Hospital_Dictionary[table_name][4]
    
    
    # If the Instrument is NOT a survey
    if Hospital_Dictionary[table_name][0] == 0:
        
        # Retrieve the appropriate columns and rows
        import_data = []
        for a in range (0, len(data)):
            auto_id = data[a][0]
            event_name = data[a][1]
            repeat_instrument = data[a][2]
            
            # Check to see if the data is filtered by repeat instrument or event_name
            if auto_id not in NoImport_List:#Do not import the data for the two participants from Hospital5859
                if Repeat_Instrument_bin == 1:
                    if repeat_instrument in StringList:
                        temp_data = data[a][0:2] + data[a][Column1:Column2]
                        import_data.append(temp_data)
                else:
                    if event_name in StringList and repeat_instrument == "":
                        temp_data = data[a][0:2] + data[a][Column1:Column2]
                        import_data.append(temp_data)
                
        # Check to see if the final columne heading is "Comlete?"
        if data[0][Column2 - 1][-8:] != "complete":
            print("Error: Last Column was not 'complete' field")

        # Import the data into the correct table in the database
        import_database(table_name, import_data)
        print("{} imported".format(table_name))
        
    # If the Instrument is a survey
    if Hospital_Dictionary[table_name][0] == 1:
        
        # Retrieve the appropriate columns and rows
        import_data = []
        questions_data = []
        for a in range (0, len(data)):
            auto_id = data[a][0]
            event_name = data[a][1]
            repeat_instrument = data[a][2]
            
            # Check to see if the data is filtered by repeat instrument or event_name
            if auto_id not in NoImport_List:#Do not import the data for the two participants from Hospital5859
                if Repeat_Instrument_bin == 1:
                    if repeat_instrument in StringList:
                        # Survey date / survey timestamp / Complete
                        temp_data = data[a][0:2] + data[a][Column1:Column1+1] + data[a][Column2-1:Column2] # Data that related to the survey
                        questions_data_row = data[a][Column1+1:Column2-1] # Survery data
                        import_data.append(temp_data)
                        questions_data.append(questions_data_row)
                else:
                    if event_name in StringList and repeat_instrument == "":
                        # Survey date / survey timestamp / Complete
                        temp_data = data[a][0:2] + data[a][Column1:Column1+1] + data[a][Column2-1:Column2] # Data that related to the survey
                        questions_data_row = data[a][Column1+1:Column2-1] # Survery data
                        import_data.append(temp_data)
                        questions_data.append(questions_data_row)

            # Gather the headings of questions of the survey
            questions_headings = data[0][Column1+1:Column2-1] 

        # Impot survey structured data into the database
        import_database_survey(table_name, import_data, questions_headings, questions_data)
        print("{} imported".format(table_name))

screen truncated
screen imported
main_pis truncated
main_pis imported
main_econsent truncated
main_econsent imported
main_econsent_declaration truncated
main_econsent_declaration imported
enrol truncated
enrol imported
randomise truncated
randomise imported
contacts truncated
contacts imported
an_hx truncated
an_hx imported
birth_hx truncated
birth_hx imported
preg_surv truncated
preg_surv imported
infection_pis truncated
infection_pis imported
infection_econsent truncated
infection_econsent imported
mri_pis truncated
mri_pis imported
mri_econsent truncated
mri_econsent imported
sleep_pis truncated
sleep_pis imported
sleep_econsent truncated
sleep_econsent imported
growth truncated
growth imported
pea_pod truncated
pea_pod imported
light_audit truncated
light_audit imported
sound_audit truncated
sound_audit imported
36w_oximetry_obs truncated
36w_oximetry_obs imported
substudies truncated
substudies imported
cardioresp truncated
cardioresp imported
headus truncated
headus imported
mri 

# CIRCADIEM FollowUp

In [330]:
# Export file from Redcap
# Data Exports, Reports, and Stats -> All data (Export data) -> check CSV / Microsoft Excel (raw) export format -> Export data *Check deidentified*

# Read the data file for import
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_FU/CIRCADIEMFU_DATA_LABELS_2023-09-27_0929.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_FU/CIRCADIEMFU_DATA_LABELS_2024-06-10_1118.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_FU/CIRCADIEMFU_DATA_LABELS_2024-07-26_1311.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_FU/CIRCADIEMFU_DATA_2024-07-31_1318.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_FU/CIRCADIEMFU_DATA_2024-08-13_1101.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_FU/CIRCADIEMFU_DATA_2024-08-20_1630.csv"
# filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_FU/CIRCADIEMFU_DATA_2024-08-26_1807.csv"


data = []
with open(r'{}'.format(filepath)) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        data.append(row)

In [331]:
# Check to see if the fields in the Hospital Instruments match those defined by the data schema.
FU_Instruments = ["fu_core_details", "fu_contacts", "fu_patient_consent_3", "fu_main_econsent", "fu_communications", "fu_withdrawal", "fu_adverse_events", "fu_growth", "fu_preg_surv", "fu_substudies", "fu_survival", "fu_asq3_2m", "fu_asq3_6m", "fu_asq3_12m", "fu_asq3_24m", "fu_asq3_27m", "fu_asq3_30m", "fu_asq3_33m", "fu_asq3_36m", "fu_asq_score", "fu_itsea", "fu_itsea_score", "fu_csbs", "fu_csbs_score", "fu_ffm", "fu_bayley4", "fu_m_int_survey", "fu_y2_surv", "fu_actigraphy_child", "fu_actigraphy_maternal", "fu_salivary_swabs_collection", "fu_bisqr", "fu_ados", "fu_sleep_diary", "fu_protocol_deviation", "fu_report_calculations"]
FU_Instruments_Survey = ["fu_patient_consent_3", "fu_main_econsent", "fu_preg_surv", "fu_asq3_2m", "fu_asq3_6m", "fu_asq3_12m", "fu_asq3_24m", "fu_asq3_27m", "fu_asq3_30m", "fu_asq3_33m", "fu_asq3_36m", "fu_itsea", "fu_csbs", "fu_m_int_survey", "fu_y2_surv", "fu_bisqr", "fu_ados", "fu_sleep_diary"]


# View all the column numbers which link to "Complete?" heading which indicates the end of a instrument's data
Column_list = [2] # Starting after the last instrument
for a in range (0 ,len(data[0])):
    temp = data[0][a]
    if "_" in temp:
        temp = temp.split("_")[-1]
    if temp == "complete": # This is the last column for the Instrument
        Column_list.append(a + 1)

for a in range (0, len(FU_Instruments)):
    
    table_name = FU_Instruments[a]
    # sql_string = """DESCRIBE {}""".format(table_name)
    # mycursor.execute(sql_string)
    # temp_data = mycursor.fetchall()

    # Column 0 = Auto ID, Column 1 = Event_Name
    # if table_name not in FU_Instruments_Survey:
    print ("!!!{},{}!!!".format(a, table_name))
    sql_field_count = 2
    for b in range (Column_list[a], Column_list[a + 1]):
        print(data[0][b])
        sql_field_count = sql_field_count + 1

    print("----------------------------------")

!!!0,fu_core_details!!!
redcap_repeat_instrument
redcap_repeat_instance
redcap_data_access_group
recr_hosp
sex
dob
edd
ga_strata
rct_other___1
rct_other___2
rct_other___3
rct_other___4
rct_other___5
rct_other___6
rct_other___7
rct_other___8
rct_other___99
rct_other___ni
rct_other___unk
rct_other___nask
rct_other___asku
rct_other___na
par_joindate
par_code
core_details_complete
----------------------------------
!!!1,fu_contacts!!!
surname
indig_yn
m_ethnicity
par2_ethnicity
par2_state
altcontact_type
contacts_complete
----------------------------------
!!!2,fu_patient_consent_3!!!
consentprono
consentspi
consentmain___1
consentmain___2
consentmain___3
consentmain___4
consentmain___ni
consentmain___unk
consentmain___nask
consentmain___asku
consentmain___na
consentmri_wa___1
consentmri_wa___2
consentmri_wa___ni
consentmri_wa___unk
consentmri_wa___nask
consentmri_wa___asku
consentmri_wa___na
consentmri_else___1
consentmri_else___ni
consentmri_else___unk
consentmri_else___nask
consentmri_e

## Create SQL Tables

### Core Details

In [332]:
# Core Details
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_core_details (
Study_ID TEXT,
Event_Name TEXT,
redcap_repeat_instrument TEXT,
redcap_repeat_instance TEXT,
redcap_data_access_group TEXT,
recr_hosp TEXT,
sex TEXT,
dob TEXT,
edd TEXT,
ga_strata TEXT,
rct_other___1 TEXT,
rct_other___2 TEXT,
rct_other___3 TEXT,
rct_other___4 TEXT,
rct_other___5 TEXT,
rct_other___6 TEXT,
rct_other___7 TEXT,
rct_other___8 TEXT,
rct_other___99 TEXT,
rct_other___ni TEXT,
rct_other___unk TEXT,
rct_other___nask TEXT,
rct_other___asku TEXT,
rct_other___na TEXT,
par_joindate TEXT,
par_code TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Contacts

In [333]:
# FU_contacts
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_contacts (
Study_ID TEXT,
Event_Name TEXT,
surname TEXT,
indig_yn TEXT,
m_ethnicity TEXT,
par2_ethnicity TEXT,
par2_state TEXT,
altcontact_type TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Patient consent 3 - Survey

In [334]:
# FU_patient_consent_3 - (Survey with no Survey Timestamp)
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_patient_consent_3 (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### Main Econsent - Survey

In [335]:
# FU_main_econsent
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_main_econsent (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### Communications

In [336]:
# FU_communications
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_communications (
Study_ID TEXT,
Event_Name TEXT,
comm_dt DATE,
comm_mode TEXT,
comm_reason TEXT,
comm_success TEXT,
comm_remind TEXT,
comm_note TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Withdrawal

In [337]:
# FU_withdrawal
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_withdrawal (
Study_ID TEXT,
Event_Name TEXT,
withdraw_yn TEXT,
withdraw_dt DATE,
Complete TEXT,
updated_at TIMESTAMP)""")

### Adverse events

In [338]:
# FU_adverse_events
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_adverse_events (
Study_ID TEXT,
Event_Name TEXT,
ae_yn___0 TEXT,
ae_yn___1 TEXT,
ae_yn___ni TEXT,
ae_yn___unk TEXT,
ae_yn___nask TEXT,
ae_yn___asku TEXT,
ae_yn___na TEXT,
ae_dtm TEXT,
ae_device___1 TEXT,
ae_device___2 TEXT,
ae_device___ni TEXT,
ae_device___unk TEXT,
ae_device___nask TEXT,
ae_device___asku TEXT,
ae_device___na TEXT,
ae_type___1 TEXT,
ae_type___2 TEXT,
ae_type___3 TEXT,
ae_type___ni TEXT,
ae_type___unk TEXT,
ae_type___nask TEXT,
ae_type___asku TEXT,
ae_type___na TEXT,
sae_yn TEXT,
ae_skin TEXT,
ae_infection TEXT,
ae_safety TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Growth

In [339]:
# FU_growth
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_growth (
Study_ID TEXT,
Event_Name TEXT,
growth_dtm DATETIME,
crown_heel TEXT,
length_cent TEXT,
length_z TEXT,
head_circ TEXT,
hc_cent TEXT,
hc_z TEXT,
wt TEXT,
wt_cent TEXT,
wt_z TEXT,
uuid_3 TEXT,
startdate_3 TEXT,
enddate_3 TEXT,
scheduledate_3 TEXT,
status_3 TEXT,
supplementaldata_3 TEXT,
serializedresult_3 TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Preg Survey - Survey

In [340]:
# FU pregnancy survey
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_preg_surv (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### Substudies

In [341]:
# FU_substudies
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_substudies (
Study_ID TEXT,
Event_Name TEXT,
sleep_ss_yn TEXT,
mri_ss_yn TEXT,
mri_ss_consent TEXT,
autism_ss_yn TEXT,
autism_ss_consent TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Survival

In [342]:
# FU_survival
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_survival (
Study_ID TEXT,
Event_Name TEXT,
dth_yn TEXT,
dth_cause TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### ASQ3_2m - Survey

In [343]:
# FU_Asq3_2m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_2m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### ASQ3_6m - Survey

In [344]:
# FU_Asq3_6m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_6m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### ASQ3_12m - Survey

In [345]:
# FU_Asq3_12m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_12m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### ASQ3_24m - Survey

In [346]:
# FU_Asq3_24m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_24m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### ASQ3_27m - Survey

In [347]:
# FU_Asq3_27m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_27m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### ASQ3_30m - Survey

In [348]:
# FU_Asq3_30m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_30m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### ASQ3_33m - Survey

In [349]:
# FU_Asq3_33m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_33m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### ASQ3_36m - Survey

In [350]:
# FU_Asq3_36m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_36m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### ASQ3_score

In [351]:
#FU_ASQ3_score
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_asq_score (
Study_ID TEXT,
Event_Name TEXT,
asq3_2m_commct TEXT,
asq3_2m_gmct TEXT,
asq3_2m_fmct TEXT,
asq3_2m_prsct TEXT,
asq3_2m_psocct TEXT,
asq3_2m_commsc TEXT,
asq3_2m_gmsc TEXT,
asq3_2m_fmsc TEXT,
asq3_2m_prssc TEXT,
asq3_2m_psocsc TEXT,
asq3_2m_totsc TEXT,
asq3_6m_commct TEXT,
asq3_6m_gmct TEXT,
asq3_6m_fmct TEXT,
asq3_6m_prsct TEXT,
asq3_6m_psocct TEXT,
asq3_6m_commsc TEXT,
asq3_6m_gmsc TEXT,
asq3_6m_fmsc TEXT,
asq3_6m_prssc TEXT,
asq3_6m_psocsc TEXT,
asq3_6m_totsc TEXT,
asq3_12m_commct TEXT,
asq3_12m_gmct TEXT,
asq3_12m_fmct TEXT,
asq3_12m_prsct TEXT,
asq3_12m_psocct TEXT,
asq3_12m_commsc TEXT,
asq3_12m_gmsc TEXT,
asq3_12m_fmsc TEXT,
asq3_12m_prssc TEXT,
asq3_12m_psocsc TEXT,
asq3_12m_totsc TEXT,
asq3_24m_commct TEXT,
asq3_24m_gmct TEXT,
asq3_24m_fmct TEXT,
asq3_24m_prsct TEXT,
asq3_24m_psocct TEXT,
asq3_24m_commsc TEXT,
asq3_24m_gmsc TEXT,
asq3_24m_fmsc TEXT,
asq3_24m_prssc TEXT,
asq3_24m_psocsc TEXT,
asq3_24m_totsc TEXT,
asq3t_2m_commct TEXT,
asq3t_2m_gmct TEXT,
asq3t_2m_fmct TEXT,
asq3t_2m_prsct TEXT,
asq3t_2m_psocct TEXT,
asq3t_2m_commsc TEXT,
asq3t_2m_gmsc TEXT,
asq3t_2m_fmsc TEXT,
asq3t_2m_prssc TEXT,
asq3t_2m_psocsc TEXT,
asq3t_2m_totsc TEXT,
asq3t_6m_commct TEXT,
asq3t_6m_gmct TEXT,
asq3t_6m_fmct TEXT,
asq3t_6m_prsct TEXT,
asq3t_6m_psocct TEXT,
asq3t_6m_commsc TEXT,
asq3t_6m_gmsc TEXT,
asq3t_6m_fmsc TEXT,
asq3t_6m_prssc TEXT,
asq3t_6m_psocsc TEXT,
asq3t_6m_totsc TEXT,
asq3t_12m_commct TEXT,
asq3t_12m_gmct TEXT,
asq3t_12m_fmct TEXT,
asq3t_12m_prsct TEXT,
asq3t_12m_psocct TEXT,
asq3t_12m_commsc TEXT,
asq3t_12m_gmsc TEXT,
asq3t_12m_fmsc TEXT,
asq3t_12m_prssc TEXT,
asq3t_12m_psocsc TEXT,
asq3t_12m_totsc TEXT,
asq3t_24m_commct TEXT,
asq3t_24m_gmct TEXT,
asq3t_24m_fmct TEXT,
asq3t_24m_prsct TEXT,
asq3t_24m_psocct TEXT,
asq3t_24m_commsc TEXT,
asq3t_24m_gmsc TEXT,
asq3t_24m_fmsc TEXT,
asq3t_24m_prssc TEXT,
asq3t_24m_psocsc TEXT,
asq3t_24m_totsc TEXT,
asq3_notes TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### ITSEA - Survey

In [352]:
#FU_ITSEA_survey
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_itsea (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### ITSEA Score

In [353]:
# FU_ITSEA_score
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_itsea_score (
Study_ID TEXT,
Event_Name TEXT,
itseacorrage TEXT,
itsea_ext_act_ct TEXT,
itsea_ext_act TEXT,
itsea_ext_agg_ct TEXT,
itsea_ext_agg TEXT,
itsea_ext_peer_ct TEXT,
itsea_ext_peer TEXT,
itsea_ext TEXT,
itsea_int_dep_ct TEXT,
itsea_int_dep TEXT,
itsea_int_fear_ct TEXT,
itsea_int_fear TEXT,
itsea_int_anx_ct TEXT,
itsea_int_anx TEXT,
itsea_int_sep_ct TEXT,
itsea_int_sep TEXT,
itsea_int_inhib_ct TEXT,
itsea_int_inhib TEXT,
itsea_internal TEXT,
itsea_dysreg_negemo_ct TEXT,
itsea_dysreg_negemo TEXT,
itsea_dysreg_sleep_ct TEXT,
itsea_dysreg_sleep TEXT,
itsea_dysreg_eat_ct TEXT,
itsea_dysreg_eat TEXT,
itsea_dysreg_sens_ct TEXT,
itsea_dysreg_sens TEXT,
itsea_dysreg_tot TEXT,
itsea_comp_compl_ct TEXT,
itsea_comp_compl TEXT,
itsea_comp_attent_ct TEXT,
itsea_comp_attent TEXT,
itsea_comp_mast_ct TEXT,
itsea_comp_mast TEXT,
itsea_comp_imit_ct TEXT,
itsea_comp_imit TEXT,
itsea_comp_emp_ct TEXT,
itsea_comp_emp TEXT,
itsea_comp_prosoc_ct TEXT,
itsea_comp_prosoc TEXT,
itsea_comp TEXT,
itsea_maladap_ct TEXT,
itsea_maladap TEXT,
itsea_socrelat_ct TEXT,
itsea_socrelat TEXT,
itsea_atyp_ct TEXT,
itsea_atyp TEXT,
itsea_individ TEXT,
bitsea_asd_prob TEXT,
bitsea_asd_comp TEXT,
itsea_asd_prob2 TEXT,
itsea_asd_comp2 TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### CSBS - Survey

In [354]:
# FU_CSBS
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_CSBS (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### CSBS Score

In [355]:
#FU_CSBS_score
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_csbs_score (
Study_ID TEXT,
Event_Name TEXT,
csbs_age TEXT,
csbs_eyegaze_rs TEXT,
csbs_comm_rs TEXT,
csbs_gestures_rs TEXT,
csbs_sounds_rs TEXT,
csbs_words_rs TEXT,
csbs_under_rs TEXT,
csbs_objuse_rs TEXT,
csbs_soccomp TEXT,
csbs_speeccomp TEXT,
csbs_symcomp TEXT,
totalscore_composite TEXT,
csbs_soccomp_sc TEXT,
csbs_speeccomp_sc TEXT,
csbs_symcomp_sc TEXT,
totalscore_stdscore TEXT,
csbs_soccomp_pr TEXT,
csbs_speeccomp_pr TEXT,
csbs_symcomp_pr TEXT,
totalscore_percentilerank TEXT,
csbs_notes TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### FFM

In [356]:
#FU_FFM
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_FFM (
Study_ID TEXT,
Event_Name TEXT,
ffm_date DATE,
ffm TEXT,
ffm_pc TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Bayley 4

In [357]:
#FU_bayley4
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_bayley4 (
Study_ID TEXT,
Event_Name TEXT,
b4_dt DATE,
b4_agem TEXT,
b4_cagem TEXT,
b4_caged TEXT,
b4_assessor TEXT,
b4_ax_loc TEXT,
b4_ax_loc_other TEXT,
b4_success_yn TEXT,
b4_incompreason TEXT,
b4_failed TEXT,
b4_failedoth TEXT,
b4_decline TEXT,
b4_declineoth TEXT,
b4_loss TEXT,
b4_nothspec TEXT,
b4_rebook_yn TEXT,
b4_lang1 TEXT,
b4_fname TEXT,
b4_parentrep TEXT,
b4_file TEXT,
b4_cg_rs TEXT,
b4_cg_ss TEXT,
b4_cog_cs TEXT,
b4_rc_rs TEXT,
b4_rc_ss TEXT,
b4_ec_rs TEXT,
b4_ec_ss TEXT,
b4_lang_sum TEXT,
b4_lang_cs TEXT,
b4_fm_rs TEXT,
b4_fm_ss TEXT,
b4_gm_rs TEXT,
b4_gm_ss TEXT,
b4_mot_sum TEXT,
b4_mot_cs TEXT,
b4_miss TEXT,
b4par_req_yn TEXT,
b4par_req_dt DATE,
b4par_sent_yn TEXT,
b4par_sent_dt DATE,
Complete TEXT,
updated_at TIMESTAMP)""")

### M_Int_Survey - Survey

In [358]:
# FU_M_Int_Survey
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_M_Int_Survey (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### 2ySurvey - Survey

In [359]:
# FU_2ySurvey
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_y2_Surv (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### Actigraphy_child

In [360]:
# FU_actigraphy_child
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_actigraphy_child (
Study_ID TEXT,
Event_Name TEXT,
act_st_dt DATE,
act_st_t TEXT,
act_end_dt DATE,
act_end_t TEXT,
act_age TEXT,
act_loc TEXT,
actname TEXT,
actagd TEXT,
act_24_yn TEXT,
sleep_24 TEXT,
sleep_n TEXT,
sleep_epoch_n TEXT,
sleep_d TEXT,
sleep_epoch_d TEXT,
dar TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Actigraphy_maternal

In [361]:
# FU_actigraphy maternal
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_actigraphy_maternal (
Study_ID TEXT,
Event_Name TEXT,
actm_st_dt DATE,
actm_st_t TEXT,
actm_end_dt DATE,
actm_end_t TEXT,
actm_loc TEXT,
actnamem TEXT,
actagdm TEXT,
actm_24_yn TEXT,
sleepm_24 TEXT,
sleepm_n TEXT,
sleepm_epoch_n TEXT,
sleepm_d TEXT,
sleepm_epoch_d TEXT,
darm TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Sample_collection

In [362]:
# FU_sample_collection
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_salivary_swabs_collection (
Study_ID TEXT,
Event_Name TEXT,
saliva_am_dtm DATETIME,
saliva_am_vol TEXT,
melat_am TEXT,
saliva_pm_dtm DATETIME,
saliva_pm_vol TEXT,
melat_pm TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### BISQR - Survey

In [363]:
# FU_BISQR
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_bisqr (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### ADOS - Survey

In [364]:
# FU_ADOS
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_ados (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### Sleep Diary - Survey

In [365]:
# FU_Sleep_diary 
mycursor.execute("""CREATE TABLE IF NOT EXISTS FU_sleep_diary (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

### Protocol Deviation

In [366]:
# FU_protocol_deviation
mycursor.execute("""CREATE TABLE IF NOT EXISTS FU_protocol_deviation (
Study_ID TEXT,
Event_Name TEXT,
devyn TEXT,
devdt DATE,
idevdt DATE,
devnotes TEXT,
devcode TEXT,
devcodeoth TEXT,
devae TEXT,
devfin TEXT,
devfindt DATE,
devethics TEXT,
devethicsdt DATE,
devsign TEXT,
devpdf TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

### Report Calculations

In [367]:
# FU_report_calculations
mycursor.execute("""CREATE TABLE IF NOT EXISTS FU_report_calculations (
Study_ID TEXT,
Event_Name TEXT,
preg_surv_sd TEXT,
preg_surv_cd TEXT,
preg_surv_dc TEXT,
preg_surv_dc2 TEXT,
sleepd_term_sd TEXT,
sleepd_term_cd TEXT,
sleepd_term_dc TEXT,
sleepd_term_dc2 TEXT,
asq3_2m_sd TEXT,
asq3_2m_cd TEXT,
asq3_2m_dc TEXT,
asq3_2m_dc2 TEXT,
mint_2m_sd TEXT,
mint_2m_cd TEXT,
mint_2m_dc TEXT,
mint_2m_dc2 TEXT,
bisqr_2m_sd TEXT,
bisqr_2m_cd TEXT,
bisqr_2m_dc TEXT,
bisqr_2m_dc2 TEXT,
sleepd_2m_sd TEXT,
sleepd_2m_cd TEXT,
sleepd_2m_dc TEXT,
sleepd_2m_dc2 TEXT,
asq3_6m_sd TEXT,
asq3_6m_cd TEXT,
asq3_6m_dc TEXT,
asq3_6m_dc2 TEXT,
mint_6m_sd TEXT,
mint_6m_cd TEXT,
mint_6m_dc TEXT,
mint_6m_dc2 TEXT,
bisqr_6m_sd TEXT,
bisqr_6m_cd TEXT,
bisqr_6m_dc TEXT,
bisqr_6m_dc2 TEXT,
sleepd_6m_sd TEXT,
sleepd_6m_cd TEXT,
sleepd_6m_dc TEXT,
sleepd_6m_dc2 TEXT,
asq3_12m_sd TEXT,
asq3_12m_cd TEXT,
asq3_12m_dc TEXT,
asq3_12m_dc2 TEXT,
mint_12m_sd TEXT,
mint_12m_cd TEXT,
mint_12m_dc TEXT,
mint_12m_dc2 TEXT,
asq3_24m_sd TEXT,
asq3_24m_cd TEXT,
asq3_24m_dc TEXT,
asq3_24m_dc2 TEXT,
mint_24m_sd TEXT,
mint_24m_cd TEXT,
mint_24m_dc TEXT,
mint_24m_dc2 TEXT,
csbs_sd TEXT,
csbs_cd TEXT,
csbs_dc TEXT,
csbs_dc2 TEXT,
itsea_sd TEXT,
itsea_cd TEXT,
itsea_dc TEXT,
itsea_dc2 TEXT,
y2s_sd TEXT,
y2s_cd TEXT,
y2s_dc TEXT,
y2s_dc2 TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

## Import Data

### Core

In [368]:
# Identify the Columns for each Instrument
FU_Instruments = ["fu_core_details", "fu_contacts", "fu_patient_consent_3", "fu_main_econsent", "fu_communications", "fu_withdrawal", "fu_adverse_events", "fu_growth", "fu_preg_surv", "fu_substudies", "fu_survival", "fu_asq3_2m", "fu_asq3_6m", "fu_asq3_12m", "fu_asq3_24m", "fu_asq3_27m", "fu_asq3_30m", "fu_asq3_33m", "fu_asq3_36m", "fu_asq_score", "fu_itsea", "fu_itsea_score", "fu_csbs", "fu_csbs_score", "fu_ffm", "fu_bayley4", "fu_m_int_survey", "fu_y2_surv", "fu_actigraphy_child", "fu_actigraphy_maternal", "fu_salivary_swabs_collection", "fu_bisqr", "fu_ados", "fu_sleep_diary", "fu_protocol_deviation", "fu_report_calculations"]

# View all the column numbers which link to "Complete?" heading which indicates the end of a instrument's data
Column_list = [2] # Starting after the last instrument
for a in range (0 ,len(data[0])):
    temp = data[0][a]
    if "_" in temp:
        temp = temp.split("_")[-1]
    if temp == "complete": # This is the last column for the Instrument
        Column_list.append(a + 1)
        
event_name_list = ["term_arm_1", "cpna1m_arm_1", "cpna2m_arm_1", "cpna3m_arm_1", "cpna35m_arm_1", "cpna6m_arm_1", "cpna12m_arm_1", "cpna22m_arm_1", "cpna24m_arm_1", "protocol_deviation_arm_1"]  

# Manually create list for each event type that triggers an instrument
Term = ["fu_core_details", "fu_contacts", "fu_main_econsent", "fu_communications", "fu_withdrawal", "fu_adverse_events", "fu_preg_surv", "fu_substudies", "fu_survival", "fu_asq_score", "fu_itsea_score", "fu_csbs_score", "fu_adverse_event", "fu_withdrawal", "fu_actigraphy_child", "fu_actigraphy_maternal", "fu_salivary_swabs_collection", "fu_sleep_diary", "fu_report_calculations"]
cPNA1m = []
cPNA2m = ["fu_growth", "fu_asq3_2m", "fu_m_int_survey", "fu_actigraphy_child", "fu_actigraphy_maternal", "fu_salivary_swabs_collection", "fu_bisqr", "fu_sleep_diary"]
cPNA3m = []
cPNA3_5m = []
cPNA6m = ["fu_growth", "fu_asq3_6m", "fu_m_int_survey", "fu_actigraphy_child", "fu_actigraphy_maternal", "fu_salivary_swabs_collection", "fu_bisqr", "fu_sleep_diary"]
cPNA12m = ["fu_growth", "fu_asq3_12m", "fu_m_int_survey"]
cPNA22m = ["fu_itsea", "fu_csbs"]
cPNA24m = ["fu_growth", "fu_asq3_24m", "fu_asq3_27m", "fu_asq3_30m", "fu_asq3_33m", "fu_asq3_36m", "fu_ffm", "fu_bayley4", "fu_m_int_survey", "fu_y2_surv", "fu_ados"]
Protocol_Deviation = ["fu_protocol_deviation"]

event_name_mapping = [Term, cPNA1m, cPNA2m, cPNA3m, cPNA3_5m, cPNA6m, cPNA12m, cPNA22m, cPNA24m, Protocol_Deviation]
       
# The name of the filter for the repeat Instruments
event_name_list_repeatInstrument = ["Sleep Diary"]

# Create a dictionary with the Hospital Instruments / If it is a survey / Starting Column / Ending Column / Event to Instrument Mapping / Repeat_Instruments

# Tables are stored into the database based on whether they are a survey or not (Manually determined)
# 1 if Survey
# 0 if Not Survey

# Manually deduce whether or not the instrument should be stored in a survey structure
FU_Instruments_Survey = ["fu_patient_consent_3", "fu_main_econsent", "fu_preg_surv", "fu_asq3_2m", "fu_asq3_6m", "fu_asq3_12m", "fu_asq3_24m", "fu_asq3_27m", "fu_asq3_30m", "fu_asq3_33m", "fu_asq3_36m", "fu_itsea", "fu_csbs", "fu_m_int_survey", "fu_y2_surv", "fu_bisqr", "fu_ados", "fu_sleep_diary"]

# The datarows are filtered either based on event_name or repeat_instrument. This will display the right data row when the data should be present
# 1 if filtered by Event Name
# 0 if filtered by Repeat Instrument

# Data rows that need to be filtered by repeat_instruments rather than event_name
Repeat_Instruments = ["fu_sleep_diary"]

FU_Dictionary = {}
# Example screen: [0, 6, 41] -> instrument: "screen", not survey, first column: 6, last column: 41
for a in range (0, len(FU_Instruments)):
    table_name = FU_Instruments[a]
    
    temp_array = []
    # Check to see if the instrument is a survey
    if table_name in FU_Instruments_Survey:
         temp_array.append(1)
    else:
         temp_array.append(0)
         
    # First column and Last column of data
    temp_array.append(Column_list[a])
    temp_array.append(Column_list[a+1])
         
    # Check to see if the instrument needs to be filtered by Repeat Instrument
    if table_name in Repeat_Instruments:
        temp_array.append(1)
         # Append an array of the repeat_instrument strings that it need to be filtered on
        temp_list = []
        for b in range (0, len(Repeat_Instruments)):
            if Repeat_Instruments[b] == table_name:
                temp_list.append(event_name_list_repeatInstrument[b])
        temp_array.append(temp_list)
    else:
        temp_array.append(0)
        # Append an array of the event_names that need to be filtered on
        temp_list = []
        for b in range (0, len(event_name_mapping)):
            if table_name in event_name_mapping[b]:
                temp_list.append(event_name_list[b])
        temp_array.append(temp_list)
        
    # Create Dictionary
    FU_Dictionary[table_name] = temp_array

In [369]:
FU_Dictionary

{'fu_core_details': [0, 2, 27, 0, ['term_arm_1']],
 'fu_contacts': [0, 27, 34, 0, ['term_arm_1']],
 'fu_patient_consent_3': [1, 34, 103, 0, []],
 'fu_main_econsent': [1, 103, 136, 0, ['term_arm_1']],
 'fu_communications': [0, 136, 143, 0, ['term_arm_1']],
 'fu_withdrawal': [0, 143, 146, 0, ['term_arm_1']],
 'fu_adverse_events': [0, 146, 174, 0, ['term_arm_1']],
 'fu_growth': [0,
  174,
  192,
  0,
  ['cpna2m_arm_1', 'cpna6m_arm_1', 'cpna12m_arm_1', 'cpna24m_arm_1']],
 'fu_preg_surv': [1, 192, 213, 0, ['term_arm_1']],
 'fu_substudies': [0, 213, 219, 0, ['term_arm_1']],
 'fu_survival': [0, 219, 222, 0, ['term_arm_1']],
 'fu_asq3_2m': [1, 222, 256, 0, ['cpna2m_arm_1']],
 'fu_asq3_6m': [1, 256, 290, 0, ['cpna6m_arm_1']],
 'fu_asq3_12m': [1, 290, 324, 0, ['cpna12m_arm_1']],
 'fu_asq3_24m': [1, 324, 359, 0, ['cpna24m_arm_1']],
 'fu_asq3_27m': [1, 359, 394, 0, ['cpna24m_arm_1']],
 'fu_asq3_30m': [1, 394, 429, 0, ['cpna24m_arm_1']],
 'fu_asq3_33m': [1, 429, 464, 0, ['cpna24m_arm_1']],
 'fu_asq

In [370]:
# Iterate through the dictionary of Instruments, Check to see if they are survey / Check to see what the rows are filtered on / Import to SQL

for table_name in FU_Dictionary:
    
    # Clear the data from the table
    sql_string = "TRUNCATE TABLE {}".format(table_name)
    mycursor.execute(sql_string)
    
    print("{} truncated".format(table_name))
    
    Survey_binary = FU_Dictionary[table_name][0]
    Column1 = FU_Dictionary[table_name][1]
    Column2 = FU_Dictionary[table_name][2]
    Repeat_Instrument_bin = FU_Dictionary[table_name][3]
    StringList = FU_Dictionary[table_name][4]
    
    
    # If the Instrument is NOT a survey
    if FU_Dictionary[table_name][0] == 0:
        
        # Retrieve the appropriate columns and rows
        import_data = []
        for a in range (0, len(data)):
            event_name = data[a][1]
            repeat_instrument = data[a][2]
            
            # Check to see if the data is filtered by repeat instrument or event_name
            if Repeat_Instrument_bin == 1:
                if repeat_instrument in StringList:
                    temp_data = data[a][0:2] + data[a][Column1:Column2]
                    import_data.append(temp_data)
            else:
                if event_name in StringList and repeat_instrument == "":
                    temp_data = data[a][0:2] + data[a][Column1:Column2]
                    import_data.append(temp_data)
                
        # Check to see if the final column heading is "Complete?"
        if data[0][Column2 - 1][-8:] != "complete":
            print("Error: Last Column was not 'complete' field")

        # Import the data into the correct table in the database
        import_database(table_name, import_data)
        print("{} imported".format(table_name))
        
    # If the Instrument is a survey
    if FU_Dictionary[table_name][0] == 1:
        
        # Retrieve the appropriate columns and rows
        import_data = []
        questions_data = []
        for a in range (0, len(data)):
            event_name = data[a][1]
            repeat_instrument = data[a][2]
            
            # Check to see if the data is filtered by repeat instrument or event_name
            if Repeat_Instrument_bin == 1:
                if repeat_instrument in StringList:
                    # Survey date / survey timestamp / Complete
                    temp_data = data[a][0:2] + data[a][Column1:Column1+1] + data[a][Column2-1:Column2] # Data that related to the survey
                    questions_data_row = data[a][Column1+1:Column2-1] # Survery data
                    import_data.append(temp_data)
                    questions_data.append(questions_data_row)
            else:
                if event_name in StringList and repeat_instrument == "":
                    # Survey date / survey timestamp / Complete
                    temp_data = data[a][0:2] + data[a][Column1:Column1+1] + data[a][Column2-1:Column2] # Data that related to the survey
                    questions_data_row = data[a][Column1+1:Column2-1] # Survery data
                    import_data.append(temp_data)
                    questions_data.append(questions_data_row)

            # Gather the headings of questions of the survey
            questions_headings = data[0][Column1+1:Column2-1] 

        # Impot survey structured data into the database
        import_database_survey(table_name, import_data, questions_headings, questions_data)
        print("{} imported".format(table_name))

fu_core_details truncated
fu_core_details imported
fu_contacts truncated
fu_contacts imported
fu_patient_consent_3 truncated
fu_patient_consent_3 imported
fu_main_econsent truncated
fu_main_econsent imported
fu_communications truncated
fu_communications imported
fu_withdrawal truncated
fu_withdrawal imported
fu_adverse_events truncated
fu_adverse_events imported
fu_growth truncated
fu_growth imported
fu_preg_surv truncated
fu_preg_surv imported
fu_substudies truncated
fu_substudies imported
fu_survival truncated
fu_survival imported
fu_asq3_2m truncated
fu_asq3_2m imported
fu_asq3_6m truncated
fu_asq3_6m imported
fu_asq3_12m truncated
fu_asq3_12m imported
fu_asq3_24m truncated
fu_asq3_24m imported
fu_asq3_27m truncated
fu_asq3_27m imported
fu_asq3_30m truncated
fu_asq3_30m imported
fu_asq3_33m truncated
fu_asq3_33m imported
fu_asq3_36m truncated
fu_asq3_36m imported
fu_asq_score truncated
fu_asq_score imported
fu_itsea truncated
fu_itsea imported
fu_itsea_score truncated
fu_itsea_score

In [371]:
# # View all the headings of the the fields separated by the "Complete" field for different forms
# for a in range (Column1,Column2):
#     temp = data[0][a].replace(" ", "_").replace("(","").replace(")","").replace("?","").replace("=", "_").replace("+", "").replace(".", "").replace("/", "").replace("-", "_").replace(":", "")
#     print(a, temp)
    
#     if temp == "Complete":
#         print("--------------------------------------------------------------------------------")

# CIRCADIEM Hospital 5859

In [372]:
# Read the data file for import ("Do not check deidentified" - it hides some of the datetimes that are needed)
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital_5859/CIRCADIEMHospital585_DATA_LABELS_2023-10-25_1743.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital_5859/CIRCADIEMHospital585_DATA_2024-08-02_1555.csv"


data = []
with open(r'{}'.format(filepath)) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        data.append(row)

In [373]:
# Check to see if the fields in the Hospital Instruments match those defined by the data schema.
Hospital_Instruments_Survey = ["preg_surv", "infection_econsent", "mri_econsent", "sleep_econsent", "preg_surv"]
Hospital_Instruments = ["screen", "enrol", "randomise", "contacts", "an_hx", "birth_hx", "preg_surv", "growth", "pea_pod", "light_audit", "sound_audit", "shift", "substudies", "cardioresp", "headus", "mri", "neomorbidity", "discharge", "survival", "adverse_event", "withdrawal"]

# View all the column numbers which link to "Complete?" heading which indicates the end of a instrument's data
Column_list = [8] # Starting after the last instrument
for a in range (0 ,len(data[0])):
    temp = data[0][a]
    if "_" in temp:
        temp = temp.split("_")[-1]
    if temp == "complete": # This is the last column for the Instrument
        Column_list.append(a + 1)

for a in range (0, len(Hospital_Instruments)):
    
    table_name = Hospital_Instruments[a]
    sql_string = """DESCRIBE {}""".format(table_name)
    mycursor.execute(sql_string)
    temp_data = mycursor.fetchall()

#    Column 0 = Auto ID, Column 1 = Event_Name
    table_name = Hospital_Instruments[a]
    print ("!!!{},{}!!!".format(a, table_name))
    for b in range (Column_list[a], Column_list[a + 1]):
        print(b, data[0][b])

    print("----------------------------------")

#     # Column 0 = Auto ID, Column 1 = Event_Name
#     if table_name not in Hospital_Instruments_Survey:
#             print ("!!!{},{}!!!".format(a, table_name))
#             sql_field_count = 2
#             for b in range (Column_list[a], Column_list[a + 1]):
#                 print(b, data[0][b],  "------------", temp_data[sql_field_count][0])
#                 sql_field_count = sql_field_count + 1

#             print("----------------------------------")
            
#     else:
#         print ("!!!{},{}!!!".format(a, table_name))
#         print("----------------------------------")

!!!0,screen!!!
8 inborn_yn
9 sex
10 b_dtm
11 b_dt
12 edd_dtm
13 edd_dt
14 ga_calc
15 ga_cat_calc
16 ga_strata
17 rct_other_yn
18 rct_other___1
19 rct_other___2
20 rct_other___3
21 rct_other___4
22 rct_other___5
23 rct_other___6
24 rct_other___7
25 rct_other___8
26 rct_other___99
27 rct_other___ni
28 rct_other___unk
29 rct_other___nask
30 rct_other___asku
31 rct_other___na
32 rctother_name
33 other_acr
34 other_neuro_yn
35 cg_neuro
36 cong_abnorm_text
37 crit_ill
38 dischhosp_exp
39 fu2y_yn
40 screen_complete
----------------------------------
!!!1,enrol!!!
41 par_consent
42 par_approach_yn
43 approach_reason___1
44 approach_reason___2
45 approach_reason___3
46 approach_reason___4
47 approach_reason___5
48 approach_reason___6
49 approach_reason___7
50 approach_reason___99
51 approach_reason___ni
52 approach_reason___unk
53 approach_reason___nask
54 approach_reason___asku
55 approach_reason___na
56 approach_other
57 enrol_dtm
58 age_enrol
59 resid_mainconsent
60 consent_upload
61 consent

## Create SQL Tables

### Shift

In [374]:
# Shift
mycursor.execute("""CREATE TABLE IF NOT EXISTS shift (
shiftdtm DATETIME,
shift_supp___1 TEXT,
shift_supp___2 TEXT,
shift_supp___3 TEXT,
shift_supp___4 TEXT,
shift_supp___5 TEXT,
shift_supp___ni TEXT,
shift_supp___unk TEXT,
shift_supp___nask TEXT,
shift_supp___asku TEXT,
shift_supp___na TEXT,
shift_o2 TEXT,
shift_hhf_fl TEXT,
shift_cpap_press TEXT,
shift_hb TEXT,
shift_ort TEXT,
shiftfio2 TEXT,
shiftspo2 TEXT,
altitude TEXT,
shift_fname TEXT,
shift_file TEXT,
spo2_24av TEXT,
fi02_24h TEXT,
shift_pred TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

## Import Data

In [375]:
# Initial 6 columns refer to linking fields
# ï»¿"Auto_ID"
# Event_Name
# Repeat_Instrument
# Repeat_Instance
# Data_Access_Group
# Survey_Identifier

# View all the column numbers which link to "Complete?" heading which indicates the end of a instrument's data
Column_list = [8] # Starting after the last instrument
for a in range (0 ,len(data[0])):
    temp = data[0][a]
    if "_" in temp:
        temp = temp.split("_")[-1]
    if temp == "complete": # This is the last column for the Instrument
        Column_list.append(a + 1)

event_name_list = ["d03_arm_1", "d7_arm_1", "d14_arm_1", "d28_arm_1", "d56_arm_1", "36w_arm_1", "disch_arm_1", "protocol_deviation_arm_1", "study_withdrawal_arm_1", "survival_arm_1"]  

# Manually create list for each event type that triggers an instrument
D03 = ["screen", "main_pis", "main_econsent", "main_econsent_declaration", "enrol", "randomise", "contacts", "an_hx", "birth_hx", "preg_surv", "growth", "headus", "report_calculations"]
D7 = ["infection_pis", "infection_econsent", "growth", "headus"]
D14 = ["growth"]
D28 = ["growth", "headus"]
D56 = ["growth"]
W36 = ["infection_pis", "infection_econsent", "mri_pis", "mri_econsent", "sleep_pis", "sleep_econsent", "growth", "pea_pod", "36w_oximetry_obs", "headus"]
Disch = ["growth", "substudies", "cardioresp", "mri", "neomorbidity", "discharge"]
Protocol_deviation = ["protocol_deviation"]
Study_withdrawal = ["withdrawal"]
Survival = ["survival"]
         
event_name_mapping = [D03, D7, D14, D28, D56, W36, Disch, Protocol_deviation, Study_withdrawal, Survival]
       
# The name of the filter for the repeat Instruments
event_name_list_repeatInstrument = ["light_audit", "sound_audit", "adverseevent"]

# Create a dictionary with the Hospital Instruments / If it is a survey / Starting Column / Ending Column / Event to Instrument Mapping / Repeat_Instruments

# Tables are stored into the database based on whether they are a survey or not (Manually determined)
# 1 if Survey
# 0 if Not Survey

# Manually deduce whether or not the instrument should be stored in a survey structure
Hospital_Instruments_Survey = ["main_econsent", "preg_surv", "infection_econsent", "mri_econsent", "sleep_econsent"]

# The datarows are filtered either based on event_name or repeat_instrument. This will display the right data row when the data should be present
# 1 if filtered by Repeat Instrument
# 0 if filtered by Event Name

# Data rows that need to be filtered by repeat_instruments rather than event_name
Repeat_Instruments = ["light_audit", "sound_audit", "adverse_event"]

Hospital_Dictionary = {}
# Example screen: [0, 6, 41] -> instrument: "screen", not survey, first column: 6, last column: 41
for a in range (0, len(Hospital_Instruments)):
    table_name = Hospital_Instruments[a]
    
    temp_array = []
    # Check to see if the instrument is a survey
    if table_name in Hospital_Instruments_Survey:
         temp_array.append(1)
    else:
         temp_array.append(0)
         
    # First column and Last column of data
    temp_array.append(Column_list[a])
    temp_array.append(Column_list[a+1])
         
    # Check to see if the instrument needs to be filtered by Repeat Instrument
    if table_name in Repeat_Instruments:
        temp_array.append(1)
         # Append an array of the repeat_instrument strings that it need to be filtered on
        temp_list = []
        for b in range (0, len(Repeat_Instruments)):
            if Repeat_Instruments[b] == table_name:
                temp_list.append(event_name_list_repeatInstrument[b])
        temp_array.append(temp_list)
    else:
        temp_array.append(0)
        # Append an array of the event_names that need to be filtered on
        temp_list = []
        for b in range (0, len(event_name_mapping)):
            if table_name in event_name_mapping[b]:
                temp_list.append(event_name_list[b])
        temp_array.append(temp_list)
        
    # Create Dictionary
    Hospital_Dictionary[table_name] = temp_array

In [376]:
Hospital_Dictionary

{'screen': [0, 8, 41, 0, ['d03_arm_1']],
 'enrol': [0, 41, 67, 0, ['d03_arm_1']],
 'randomise': [0, 67, 78, 0, ['d03_arm_1']],
 'contacts': [0, 78, 122, 0, ['d03_arm_1']],
 'an_hx': [0, 122, 158, 0, ['d03_arm_1']],
 'birth_hx': [0, 158, 175, 0, ['d03_arm_1']],
 'preg_surv': [1, 175, 197, 0, ['d03_arm_1']],
 'growth': [0,
  197,
  209,
  0,
  ['d03_arm_1',
   'd7_arm_1',
   'd14_arm_1',
   'd28_arm_1',
   'd56_arm_1',
   '36w_arm_1',
   'disch_arm_1']],
 'pea_pod': [0, 209, 235, 0, ['36w_arm_1']],
 'light_audit': [0, 235, 241, 1, ['light_audit']],
 'sound_audit': [0, 241, 247, 1, ['sound_audit']],
 'shift': [0, 247, 272, 0, []],
 'substudies': [0, 272, 293, 0, ['disch_arm_1']],
 'cardioresp': [0, 293, 319, 0, ['disch_arm_1']],
 'headus': [0,
  319,
  329,
  0,
  ['d03_arm_1', 'd7_arm_1', 'd28_arm_1', '36w_arm_1']],
 'mri': [0, 329, 332, 0, ['disch_arm_1']],
 'neomorbidity': [0, 332, 359, 0, ['disch_arm_1']],
 'discharge': [0, 359, 369, 0, ['disch_arm_1']],
 'survival': [0, 369, 375, 0, 

In [377]:
# Iterate through the dictionary of Instruments, Check to see if they are survey / Check to see what the rows are filtered on / Import to SQL

for table_name in Hospital_Dictionary:
    
    Survey_binary = Hospital_Dictionary[table_name][0]
    Column1 = Hospital_Dictionary[table_name][1]
    Column2 = Hospital_Dictionary[table_name][2]
    Repeat_Instrument_bin = Hospital_Dictionary[table_name][3]
    StringList = Hospital_Dictionary[table_name][4]
    
    # If the Instrument is NOT a survey
    if Hospital_Dictionary[table_name][0] == 0:
        
        # Retrieve the appropriate columns and rows
        import_data = []
        for a in range (0, len(data)):
            event_name = data[a][1]
            repeat_instrument = data[a][2]
            
            # Check to see if the data is filtered by repeat instrument or event_name
            if Repeat_Instrument_bin == 1:
                if repeat_instrument in StringList:
                    temp_data = data[a][0:2] + data[a][Column1:Column2]
                    import_data.append(temp_data)
            else:
                if event_name in StringList and repeat_instrument == "":
                    temp_data = data[a][0:2] + data[a][Column1:Column2]
                    import_data.append(temp_data)
                    
        # Manually Import Enrol Table
        if table_name == "enrol":
            import_data = []
            for a in range (0, len(data)):
                event_name = data[a][1]
                repeat_instrument = data[a][2]

                if event_name in StringList and repeat_instrument == "":
                    temp_data = data[a][0:2] + data[a][41:57] + [None, None] + data[a][59:69]
                    import_data.append(temp_data)
                    
        if table_name == "contacts":
            import_data = []
            for a in range (0, len(data)):
                event_name = data[a][1]
                repeat_instrument = data[a][2]

                if event_name in StringList and repeat_instrument == "":
                    # There are alot of identifiable fields that need to be filtered out
                    temp_data = data[a][0:2] + [data[a][82]] + [data[a][86]] + [data[a][94]] + [data[a][97]] + [data[a][98]] + [data[a][102]] + [data[a][108]] + [data[a][121]]
                    import_data.append(temp_data)
                    
        if table_name == "growth":
            import_data = []
            for a in range (0, len(data)):
                event_name = data[a][1]
                repeat_instrument = data[a][2]

                if event_name in StringList and repeat_instrument == "":
                    temp_data = data[a][0:2] + [None] + data[a][197:209] #Missing growth_timestamp
                    import_data.append(temp_data)
                    
        if table_name == "substudies":
            import_data = []
            for a in range (0, len(data)):
                event_name = data[a][1]
                repeat_instrument = data[a][2]

                if event_name in StringList and repeat_instrument == "":
                    temp_data = data[a][0:2] + data[a][272:287] + [None]*5 + data[a][287:293] #Missing sleep_yn - sleep_consent_file
                    import_data.append(temp_data)
                    
        if table_name == "cardioresp":
            import_data = []
            for a in range (0, len(data)):
                event_name = data[a][1]
                repeat_instrument = data[a][2]

                if event_name in StringList and repeat_instrument == "":
                    temp_data = data[a][0:2] + [None] + data[a][293:319] #Missing cardioresp_timestamp
                    import_data.append(temp_data)
                    
        if table_name == "neomorbidity":
            import_data = []
            for a in range (0, len(data)):
                event_name = data[a][1]
                repeat_instrument = data[a][2]

                if event_name in StringList and repeat_instrument == "":
                    temp_data = data[a][0:2] + [None] + data[a][332:359] #Missing neomorbidity_timestamp
                    import_data.append(temp_data)   
                    
        if table_name == "discharge":
            import_data = []
            for a in range (0, len(data)):
                event_name = data[a][1]
                repeat_instrument = data[a][2]

                if event_name in StringList and repeat_instrument == "":
                    temp_data = data[a][0:2] + [None] + data[a][359:363] + [None]*13 + data[a][363:369] #Missing discharge dcg2 - 5
                    import_data.append(temp_data)
                    
        if table_name == "adverse_event":
            import_data = []
            for a in range (0, len(data)):
                event_name = data[a][1]
                repeat_instrument = data[a][2]

                if event_name in StringList and repeat_instrument == "":
                    temp_data = data[a][0:2] + data[a][375:376] + [None]*2 + data[a][376:377] + [None]*3 + data[a][377:384] + [None]*2 + data[a][384:398] + [None]*3 + data[a][398:399] + [None] + data[a][399:400] 
                    import_data.append(temp_data)
                    
        if table_name == "withdrawal":
            import_data = []
            for a in range (0, len(data)):
                event_name = data[a][1]
                repeat_instrument = data[a][2]

                if event_name in StringList and repeat_instrument == "":
                    temp_data = data[a][0:2] + data[a][400:401] + data[a][402:405]
                    import_data.append(temp_data)
                
        # Check to see if the final columne heading is "Comlete?"
        if data[0][Column2 - 1][-8:] != "complete":
            print("Error: Last Column was not 'complete' field")

        # Import the data into the correct table in the database
        import_database(table_name, import_data)
        print("{} imported".format(table_name))
        
    # If the Instrument is a survey
    if Hospital_Dictionary[table_name][0] == 1:
        
        # Retrieve the appropriate columns and rows
        import_data = []
        questions_data = []
        for a in range (0, len(data)):
            event_name = data[a][1]
            repeat_instrument = data[a][2]
            
            # Check to see if the data is filtered by repeat instrument or event_name
            if Repeat_Instrument_bin == 1:
                if repeat_instrument in StringList:
                    # Survey date / survey timestamp / Complete
                    temp_data = data[a][0:2] + data[a][Column1:Column1+1] + data[a][Column2-1:Column2] # Data that related to the survey
                    questions_data_row = data[a][Column1+1:Column2-1] # Survery data
                    import_data.append(temp_data)
                    questions_data.append(questions_data_row)
            else:
                if event_name in StringList and repeat_instrument == "":
                    # Survey date / survey timestamp / Complete
                    temp_data = data[a][0:2] + data[a][Column1:Column1+1] + data[a][Column2-1:Column2] # Data that related to the survey
                    questions_data_row = data[a][Column1+1:Column2-1] # Survery data
                    import_data.append(temp_data)
                    questions_data.append(questions_data_row)

            # Gather the headings of questions of the survey
            questions_headings = data[0][Column1+1:Column2-1] 

        # Impot survey structured data into the database
        import_database_survey(table_name, import_data, questions_headings, questions_data)
        print("{} imported".format(table_name))

screen imported
enrol imported
randomise imported
contacts imported
an_hx imported
birth_hx imported
preg_surv imported
growth imported
pea_pod imported
light_audit imported
sound_audit imported
shift imported
substudies imported
cardioresp imported
headus imported
mri imported
neomorbidity imported
discharge imported
survival imported
adverse_event imported
withdrawal imported


In [378]:
print("Data Import Successful")

Data Import Successful


In [379]:
# Release Memory
%reset -f